In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import json
pd.set_option('display.max_rows', 1000)
%matplotlib inline

In [ ]:
# Read config
with open('config.json') as json_file:
    f = json.load(json_file)
    NUMBER_OF_QUERIES = f['number_of_queries']
    REPETITIONS = f['repetitions']
    ROWS = f['rows']
    PARTITION_SIZE = f['partition_size']
    PROGRESSIVE_INDEX_DELTAS = f['deltas']

# Algorithms and Experiments defitions
algorithms = {
    'AverageKDTree': {
        'name': 'average_kd_tree',
        'color': 'red',
        'dash': 'dot',
        'show_name': 'AvgKD',
        'type': 'full_index',
        'delta': '0.0',
        'partition_size': PARTITION_SIZE
    },
    'MedianKDTree': {
        'name': 'median_kd_tree',
        'color': 'red',
        'dash': 'solid',
        'show_name': 'MedKD',
        'type': 'full_index',
        'delta': '0.0',
        'partition_size': PARTITION_SIZE
    },
    'CrackingKDTree': {
        'name': 'cracking_kd_tree',
        'color': 'green',
        'dash': 'dot',
        'show_name': 'AKD',
        'type': 'adaptive',
        'delta': '0.0',
        'partition_size': PARTITION_SIZE
    },
    'Quasii': {
        'name': 'quasii',
        'color': 'green',
        'dash': 'solid',
        'show_name': 'Q',
        'type': 'adaptive',
        'delta': '0.0',
        'partition_size': PARTITION_SIZE
    },
    'FullScan': {
        'name': 'full_scan_cl',
        'color': 'black',
        'dash': 'dot',
        'show_name': 'FS',
        'type': 'full_index',
        'partition_size': '0',
        'delta':'0.0',
    },
    'ProgressiveIndexCostModel': {
        'name': 'progressive_index_cm',
        'color': 'purple',
        'dash': 'dot',
        'show_name': 'GPKD(.2)',
        'type': 'adaptive',
        'delta': PROGRESSIVE_INDEX_DELTAS[0],
        'partition_size': PARTITION_SIZE
    },
    'ProgressiveIndex': {
        'name': 'progressive_index',
        'color': 'purple',
        'dash': 'solid',
        'show_name': 'PKD(.2)',
        'type': 'adaptive',
        'delta': PROGRESSIVE_INDEX_DELTAS[0],
        'partition_size': PARTITION_SIZE
    }
}

for key, value in algorithms.items():
    name = value['name']
    delta = value['delta']
    partition_size = value['partition_size']
    algorithms[key]['alg_id'] = f"{name}-{delta}-{partition_size}"
    
# Real World Experiments
experiments = {
    'Power': {
        'name': 'Power',
        'name-in-file': 'power',
        'sel': '0.0',
        'n_rows': ROWS,
        'n_queries': NUMBER_OF_QUERIES,
        'base_folder': 'results'
    },
    'Skyserver': {
        'name': 'Skyserver',
        'name-in-file': 'skyserver',
        'n_rows': '0',
        'n_queries': '0',
        'sel': '0.0',
        'base_folder': 'results'
    },
    'Genomics Mixed': {
        'name': f'Genomics',
        'name-in-file': 'genomics_query_8',
        'n_rows': ROWS,
        'n_queries': '100',
        'sel': '0.0',
        'base_folder': 'results'
    }
}

for key, value in experiments.items():
    name = value['name-in-file']
    rows = value['n_rows']
    n_queries = value['n_queries']
    sel = value['sel']
    experiments[key]['exp_id'] = f"{name}-{rows}-{n_queries}-{sel}"

In [ ]:
# Input/Output
def read(alg, exp):
    if exp.startswith('Shifting'):
        n_queries_per_run = 10
        uni = read(alg, 'Uniform' + experiments[exp]['n_cols'])
        n_runs = int(len(uni)/n_queries_per_run) - 1
        df_final = uni.head(n_queries_per_run)
        for _ in range(int(n_runs)):
            temp = uni.head(n_queries_per_run).copy()
            df_final = df_final.append(temp, ignore_index=True)
    else:
        df = pd.read_csv(f"results/{algorithms[alg]['alg_id']}-{experiments[exp]['exp_id']}.csv")
        repetitions = df['repetition'].max() + 1
        step = int(len(df.index)/repetitions)
        df_final = df[:step].copy().reset_index()
        for rep in range(1, repetitions):
            df_final += df[step * (rep) : step * (rep + 1)].copy().reset_index()

        df_final = df_final/repetitions
    
    if 'index_search_time' not in df_final:
        df_final['index_search_time'] = 0.0
    if 'tuples_scanned' not in df_final:
        df_final['tuples_scanned'] = 0.0
    if 'number_of_nodes' not in df_final:
        df_final['number_of_nodes'] = 0.0
    df_final['query_time'] = df_final['initialization_time'] + df_final['index_search_time'] + df_final['scan_time'] + df_final['adaptation_time']
    df_final['query_time_cumsum'] = df_final['query_time'].cumsum()
    return df_final

                     
def read_multiple(algs, exp):
    ''' Reads multiple algorithms in an experiment, return three arrays: dfs, colors, names
    '''
    dfs = []
    colors = []
    names = []
    dashes = []
    for alg in algs:
        dfs.append(read(alg, exp))
        names.append(algorithms[alg]['show_name'])
        colors.append(algorithms[alg]['color'])
        dashes.append(algorithms[alg]['dash'])
    
    return dfs, colors, dashes, names,

In [ ]:
# Helper methods
def get_first_query(df):
    return df['query_time'].iloc[0]

def get_payoff(df, baseline):
    p = [i for i, x in enumerate(df['query_time_cumsum'] - (baseline['query_time_cumsum'])) if x > 0]
    if len(p) == 0:
        return len(df)
    return p[-1]

def get_convergence(df, df_type=''):
    if df_type == 'full_index':
        return 0
    c = [i for i, x in enumerate(df['adaptation_time']) if x != 0.0]
    if(len(c) == 0):
        return len(c)
    else:
        return c[-1]

def get_robustness(df, df_type=''):
    if df_type == 'full_index':
        return 0
    return np.var(df['query_time'][:min(50, get_convergence(df, df_type))])

def get_total_time(df, lower=0, upper=-1):
    return df['query_time'][lower:upper].sum()

In [ ]:
# Figures
def create_figure(data=[]):
    fig = go.Figure(
        data=data,
        layout=go.Layout(
#             width=1500,
            plot_bgcolor='rgba(0,0,0,0)',
            font=dict(
                size=42
            ),
            yaxis=dict(
                showline=True,
                linewidth=2,
                linecolor='black',
                showgrid=True,
                gridwidth=1,
                gridcolor='lightgrey',
                zeroline=False,
                zerolinecolor='rgba(0, 0, 0, 0)',
                ticks="inside",
                ticklen=5
            ),
            xaxis=dict(
                showline=True,
                linewidth=2,
                linecolor='black',
                ticks='inside',
                zeroline=True,
                ticklen=5
            ),
            legend=dict(
                font=dict(
                    size=30,
                    color="black"
                )
            )
        )
    )
    return fig

def line(exp, algs, attr, limit=2000):
    dfs, colors, dashes, names = read_multiple(algs, exp)
    fig = create_figure()
    
    lines = []
    
    biggest = 0
    
    for i, df in enumerate(dfs):
        per_query_times = np.array(df[attr][:limit])
        if biggest < np.max(per_query_times):
            biggest = np.max(per_query_times)
        lines.append(
            go.Scatter(
                name=names[i],
                x=list(range(len(per_query_times))),
                y=per_query_times,
#                 marker_color=colors[i],
                mode='lines',
                line=dict(width=4, dash=dashes[i])
            )
        )
    
    fig.add_traces(data=lines)
    fig.update_layout(showlegend=True, yaxis_title=attr)
    return fig

def workload_selectivity(exp):
    fig = create_figure()
    df = read('FullScan', exp)
    sel = ((df['tuples_scanned']/df['scan_overhead'])/df['tuples_scanned']) * 100
    
    fig.add_traces(
        data=go.Scatter(
            name='selectivity',
            x=list(range(len(sel))),
            y=sel,
            mode='lines',
            line=dict(width=4)
        )
    )
    fig.update_layout(showlegend=True, yaxis_title='Selectivity (%)')
    return fig


def per_query(exp, algs, limit=1000):
    dfs, colors, dashes, names = read_multiple(algs, exp)
    fig = create_figure()
    
    lines = []
    
    biggest = 0
    
    for i, df in enumerate(dfs):
        alg = algs[i]
        per_query_times = np.array(df['query_time'][:limit]) * 1000
        if biggest < np.max(per_query_times):
            biggest = np.max(per_query_times)
        lines.append(
            go.Scatter(
                name=names[i],
                x=list(range(len(per_query_times))),
                y=per_query_times,
                marker_color=colors[i],
                mode='lines',
                line=dict(width=6, dash=dashes[i])
            )
        )
        
    fig.add_traces(data=lines)
    fig.update_layout(showlegend=True, yaxis_title='Time (milliseconds)')
    fig.update_layout(legend_orientation="h", legend=dict(x=.25, y=1.2))
    fig.update_yaxes(type="log")
    return fig

def cummulative(exp, algs, limit=5000):
    dfs, colors, dashes, names = read_multiple(algs, exp)
    fig = create_figure()
    
    lines = []
    
    biggest = 0
    
    for i, df in enumerate(dfs):
        alg = algs[i]
        per_query_times = np.array(df['query_time_cumsum'][:limit])
        lines.append(
            go.Scatter(
                name=names[i],
                x=list(range(len(per_query_times))),
                y=per_query_times,
                marker_color=colors[i],
                mode='lines',
                line=dict(width=6, dash=dashes[i])
            )
        )
    
    fig.add_traces(data=lines)
    fig.update_layout(legend_orientation="h", legend=dict(x=.25, y=1.2))
    fig.update_layout(showlegend=True, yaxis_title='Time (Seconds)')
    return fig

def number_of_nodes(exp, algs, limit=5000):
    dfs, colors, dashes, names = read_multiple(algs, exp)
    fig = create_figure()
    
    lines = []
    
    biggest = 0
    
    for i, df in enumerate(dfs):
        per_query_times = np.array(df['number_of_nodes'][:limit])
        if biggest < np.max(per_query_times):
            biggest = np.max(per_query_times)
        lines.append(
            go.Scatter(
                name=names[i],
                x=list(range(len(per_query_times))),
                y=per_query_times,
                marker_color=colors[i],
                mode='lines',
                line=dict(width=6, dash=dashes[i])
            )
        )
    
    fig.add_traces(data=lines)
    fig.update_layout(legend_orientation="h", legend=dict(x=.25, y=1.2))
    fig.update_layout(showlegend=True, yaxis_title='# Nodes')
    return fig

def tuples_scanned(exp, algs, limit=5000):
    dfs, colors, dashes, names = read_multiple(algs, exp)
    fig = create_figure()
    
    lines = []
    
    
    for i, df in enumerate(dfs):
        per_query_times = np.array(df['tuples_scanned'][:limit])
        lines.append(
            go.Scatter(
                name=names[i],
                x=list(range(len(per_query_times))),
                y=per_query_times,
                marker_color=colors[i],
                mode='lines',
                line=dict(width=6, dash=dashes[i])
            )
        )
    
    fig.add_traces(data=lines)
    fig.update_layout(legend_orientation="h", legend=dict(x=.25, y=1.2))
    fig.update_layout(showlegend=True, yaxis_title='Tuples Scanned')
    return fig

def break_down(exp, algs, limit):
    dfs, _, _, names = read_multiple(algs, exp)
    initializations = np.array([x['initialization_time'][:limit].sum() for x in dfs])
    adaptation = np.array([x['adaptation_time'][:limit].sum() for x in dfs])
    search = np.array([x['index_search_time'][:limit].sum() for x in dfs])
    scan = np.array([x['scan_time'][:limit].sum() for x in dfs])
    
#     fig = create_figure(data=[
#         go.Bar(name='Initialization', x=names, y=initializations, marker_color='rgb(176, 201, 146)'),
#         go.Bar(name='Adaptation', x=names, y=adaptation, marker_color='rgb(195, 114, 28)'),
#         go.Bar(name='Index Search', x=names, y=search, marker_color='rgb(197, 255, 114)'),
#         go.Bar(name='Scan', x=names, y=scan, marker_color='rgb(237, 218, 123)'),
#     ])

    scan_color = '#AB63FA'
    index_search_color = '#03CC96'
    adaptation_color = '#EF553B'
    initialization_color = '#636EFA'

    data = [
        go.Bar(name='Initialization', x=names, y=initializations, width=0.5, marker_color=initialization_color, showlegend=False),
        go.Bar(name='Adaptation', x=names, y=adaptation, width=0.5, marker_color=adaptation_color, showlegend=False),
        go.Bar(name='Index Search', x=names, y=search, width=0.5, marker_color=index_search_color,showlegend=False),
        go.Bar(name='Scan', x=names, y=scan, width=0.5, marker_color=scan_color,showlegend=False),
    ]
    
    data.append(
        go.Scatter(
            name='Initialization',
            x=[None],
            y=[None],
            mode='markers',
            marker_color=initialization_color,
            marker=dict(
                size=40,
                symbol='square'
            ),
        )
    )
    
    data.append(
        go.Scatter(
            name='Adaptation',
            x=[None],
            y=[None],
            mode='markers',
            marker_color=adaptation_color,
            marker=dict(
                size=40,
                symbol='square'
            ),
        )
    )
    
    data.append(
        go.Scatter(
            name='Index Search',
            x=[None],
            y=[None],
            mode='markers',
            marker_color=index_search_color,
            marker=dict(
                size=40,
                symbol='square'
            ),
        )
    )
    
    data.append(
        go.Scatter(
            name='Scan',
            x=[None],
            y=[None],
            mode='markers',
            marker_color=scan_color,
            marker=dict(
                size=40,
                symbol='square'
            ),
        )
    )

    fig = create_figure(data=data)
    
    # Change the bar mode
    fig.update_layout(barmode='stack')
    fig.update_layout(legend_orientation="h", legend=dict(x=.25, y=1.2))
    fig.update_layout(yaxis_title='Time (seconds)')
    return fig

In [ ]:
# Latex Tables

def pandas_to_latex(df, highlight='min', ignore_last=False, float_format="%.2f"):
    mins = []
    maxs = []
    for i in range(len(df)):
        row = df.iloc[i]
        c_min = float('inf')
        c_max = -float('inf')
        if ignore_last:
            length = len(row) - 1
        else:
            length = len(row)
        for j in range(length):
            if row[j] == '-' or row[j] == '*':
                continue
            if c_min > float(row[j]):
                c_min = float(row[j])
            if c_max < float(row[j]):
                c_max = float(row[j])
        mins.append(c_min)
        maxs.append(c_max)
    
    for i in range(len(df)):
        row = df.iloc[i]
        for j in range(len(row)):
            if row[j] == '-' or row[j] == '*':
                continue
            if highlight == 'min':
                if float(row[j]) == mins[i]:
                    row[j] = "\cellcolor{green!25}" + (float_format % float(row[j]))
                else:
                    row[j] = float_format % float(row[j])
            if highlight == 'max':
                if float(row[j]) == maxs[i]:
                    row[j] = "\cellcolor{green!25}" + (float_format % float(row[j]))
                else:
                    row[j] = float_format % float(row[j])
    
    return df.to_latex(multicolumn=True, multirow=True, escape=False)

def metrics(exps, algs):
    '''
    ||||||||||||Algorithm 1| Algorithm 2|...
    First Query|   1.11.   |     15.    |...
    ...
    '''
    data = {}

    # create indexes
    index_exp = []
    index_metric = []
    
    
    metrics = ['First Query', 'PayOff', 'Convergence', 'Robustness', 'Time']

    
    for exp in exps:
        dfs, _, _, names = read_multiple(algs, exp)
        
        # initialize the data dict with empty arrays for each algorithm
        for name in names:
            if name not in data:
                data[name] = []

        baseline = read('FullScan', exp)

        index_exp += [experiments[exp]['name']] * len(metrics)
        index_metric += metrics
        
        for df, name, alg in zip(dfs, names, algs):
            data[name].append('%.2f' % get_first_query(df))
            po = get_payoff(df, baseline)
            if po == len(df):
                data[name].append('-')
            else:
                data[name].append(po)
            conv = get_convergence(df, algorithms[alg]['type'])
            if conv == 0:
                data[name].append('-')
            elif conv >= len(df)-1:
                data[name].append('*')
            else:
                data[name].append(conv)
            
            robust = get_robustness(df, algorithms[alg]['type'])
            if robust == 0:
                data[name].append('-') 
            else:
                data[name].append('%.E' % robust)
            
            
            data[name].append('%.2f' %get_total_time(df))

    index = [index_exp, index_metric]
    df = pd.DataFrame(data, index=index)

    return df
    
    latex = df.to_latex(multicolumn=True, multirow=True)

    for exp in exps:
        latex = latex.replace(exp, "\\rotatebox[origin=c]{90}{%s}" % exp)
    return latex

In [ ]:
exps = ['Power', 'Genomics Mixed', 'Skyserver']
m = metrics(exps, [
        'MedianKDTree',
        'AverageKDTree',
        'Quasii',
        'CrackingKDTree',
        'ProgressiveIndex',
        'ProgressiveIndexCostModel',
        'FullScan',
])
m

# First Query Analysis

In [ ]:
idx = pd.IndexSlice
a = m.loc[idx[:, 'First Query'], :]
a = a.reset_index(level=1, drop=True)
print(pandas_to_latex(a, 'min', ignore_last=True))
a

# Payoff Analysis

In [ ]:
idx = pd.IndexSlice
a = m[['AvgKD', 'MedKD', 'AKD', 'Q', 'PKD(.2)', 'GPKD(.2)']].loc[idx[:, 'PayOff'], :]
a = a.reset_index(level=1, drop=True)
print(pandas_to_latex(a, 'min', ignore_last=False, float_format="%.0f"))
a

### Robustness

In [ ]:
idx = pd.IndexSlice
a = m[['AKD', 'Q', 'PKD(.2)', 'GPKD(.2)']].loc[idx[:, 'Robustness'], :]
a = a.reset_index(level=1, drop=True)
print(pandas_to_latex(a, 'min', ignore_last=False, float_format='%.E'))
a

# Response time

In [ ]:
idx = pd.IndexSlice
a = m.loc[idx[:, 'Time'], :]
a = a.reset_index(level=1, drop=True)
print(pandas_to_latex(a, 'min', ignore_last=False, float_format="%.1f"))
a